# ENVT3031 Practical 1: Accessing TERN Surveillance data with R

This tutorial is modified from the "'AusplotsR' Package and AusPlots Data Basics"

* Blanco-Martin, Bernardo (2019). Tutorial: "'AusplotsR' Package and AusPlots Data Basics". Terrestrial Ecology Research Network. Version 2019.06.0, June 2019. https://github.com/ternaustralia/TERN-Data-Skills/tree/master/EcosystemSurveillance_PlotData/AusPplots_BasicTutorial

`ausplotsR` is an R package for live extraction and preparation of TERN AusPlots ecosystem monitoring data. Through `ausplotsR`, users can: 
* Directly obtain plot-based data on vegetation and soils across Australia
* Preprocess these data into structures that facilitate the visualisation and analysis of ausplots data

Data preprocessing includes the computation of:
* Species occurrence
* Vegetation fractional and single cover
* Growth form
* Basal area (see below for details)



### The `ausplotsR` package currently includes 6 functions:

* `get_ausplots`: Extracts AusPlots data in R. The stating point for any AusPlots data exploration and analysis in R. 
* `species_table`: Generates species occurrence matrices using the chosen scoring method (i.e. presence/absence, cover, frequencey, or IVI index) from a data frame of individual raw intercept hits (generated from AusPlots data using the `get_ausplots` function).
* `fractional_cover`: Calculates fractional cover (i.e., the proportional cover of green vegetation, dead vegetation and bare substrate) from a data frame of individual raw intercept hits (generated from AusPlots data using the `get_ausplots` function).
* `growth_form_table`: Generates occurrence matrices for NVIS plant growth forms in plots using the chosen scoring method (i.e. presence/absence, percent cover or species richness -number of species assigned to a particular growth form-) from a data frame of individual raw intercept hits (generated from AusPlots data using the `get_ausplots` function).
* `single_cover_value`: Calculates a total vegetation cover by height and/or growh form per site from a data frame of individual raw intercept hits (generated from AusPlots data using the `get_ausplots` function). In this fucntion cover can be subsetted to vegetation over a specified height and/or by plant growth forms. By default, vegetation cover is calculated per plot for tree growth forms of 5 metres or higher (i.e. forests).
* `basal_area`: Calculates basal area (or number of basal wedge hits) for each plot using the raw basal wedge data (generated from AusPlots data using the `get_ausplots` function).


# INSTALLING and LOADING `ausplotsR`
R libraries need to be installed and loaded before they can be used in the R environment. 
### Installing
However, we are running R within the ecocloud enviornment which already has 'ausplotsR' installed. So we do not have to install it in this instance. If you are trying to complete these excersises in a different computer environment I suggest you see the examples in the "'AusplotsR' Package and AusPlots Data Basics" (see link above).
### Loading 'ausplotsR'
All libraries that are used in R must be loaded before they are available. This is achieved through the simple command below.

In [1]:
## Load the package
library(ausplotsR)

Loading required package: plyr
Loading required package: R.utils
Loading required package: R.oo
Loading required package: R.methodsS3
R.methodsS3 v1.7.1 (2016-02-15) successfully loaded. See ?R.methodsS3 for help.
Registered S3 method overwritten by 'R.oo':
  method        from       
  throw.default R.methodsS3
R.oo v1.22.0 (2018-04-21) successfully loaded. See ?R.oo for help.

Attaching package: ‘R.oo’

The following objects are masked from ‘package:methods’:

    getClasses, getMethods

The following objects are masked from ‘package:base’:

    attach, detach, gc, load, save

R.utils v2.9.0 successfully loaded. See ?R.utils for help.

Attaching package: ‘R.utils’

The following object is masked from ‘package:utils’:

    timestamp

The following objects are masked from ‘package:base’:

    cat, commandArgs, getOption, inherits, isOpen, nullfile, parse,
    warnings

Loading required package: simba
Loading required package: vegan
Loading required package: permute

Attaching package: 

Help on the `ausplotsR` package and a vignette with a guide on how to use the package can be obtained with the code below

In [ ]:
#help(ausplotsR)
#browseVignettes(package="ausplotsR")

# Part 1: OBTAIN & EXPLORE AusPlots DATA: `get_ausplots` function

The `get_ausplots` function extracts and compiles AusPlots data. 

Data of specific types, sites, geographical locations, and/or species can be requested via the function arguments.

*DATA TYPES:* Up to 8 different types of data can be obtained by setting the corresponding arguments to TRUE/FALSE:

  * `site_info`: Site summary data. Includes (among others): plot and visit details, landform data, geographic coordinates, and notes. Included by default. Site summary data are stored in the `site.info` data frame.
  * `structural_summaries`: Site vegetation structural summaries. Site vegetation structural summary data are stored in the `struct.summ` data frame.
  * `veg.vouchers`: Complete set of species records for the plot determined by a herbarium plus ID numbers for silica-dried tissue samples. Included by default. Vegetation vouchers data are stored in the `veg.vouch` data frame. 
  * `veg.PI`: Point Intercept (PI) data. Includes data on: substrate, plant species, growth form and height, etc at each of (typically) 1010 points per plot. Included by default. Vegetation point intercept data are stored in the `veg.PI` data frame.
  * `basal.wedge`: Basal Wedge Data Raw Hits. These data are required for the calculation of Basal Area by Species by Plot. Basal wedge data are stored in the `veg.basal` data frame.
  * `soil_subsites`: Information on what soil and soil metagenomics samples were taken at nine locations across the plot and their identification barcode numbers. Soil and soil metagenomics data are stored in the `soil.subsites` data frame. 
  * `soil_bulk_density`: Soil bulk density. Soil bulk density data are stored in the `soil.bulk` data frame.
  * `soil_character`: Soil characterisation and sample ID data at 10 cm increments to a depth of 1 m. Soil characterisation and sample ID data are stored in the `soil.char` data frame.

*SPATIAL FILTERING:* AusPlot data can be spatially subset via the `get_ausplots` function arguments in two ways:

  * `my.Plot_IDs`: Character vector with the plots IDs of specific AusPlots plots. 
  * `bounding_box`: Spatial filter for selecting AusPlots based on a rectangular box, in the format of e.g. c(xmin, xmax, ymin, ymax). AusPlots spatial data are are in longlat, thus x is the longitude and y is the latitude of the box/extent object (e.g., c(120, 140, -30, -10)).  

## Example 1: In this example we download all the available data at three ausplot sites

* The following code makes an extracts all availalbe data from the database for three sites in SA, Qld. and the NT
* The code puts the extracted data into the 'list object' called 'AP.data'
* AP.data contains a series of data frames (fancy R tables) that we will explore in the rest of the practical

In [2]:
# Obtain the data ('site_info', 'veg.vouchers', and 'veg.PI' are retrieve by default)
AP.data = get_ausplots( my.Plot_IDs=c("SATFLB0004", "QDAMGD0022", "NTASTU0002"),
                        structural_summaries=TRUE, basal.wedge=TRUE,
                        soil_subsites=TRUE, soil_bulk_density=TRUE, soil_character=TRUE  )

User-supplied Plot_IDs located. 


* Explore retrieved data by running the subsequent cells and thinking about what is returned

In [ ]:
# By typeing class(AP.data) we are told that the data structure is a 'list'
class(AP.data)

In [ ]:
# By running 'summary(AP.data)' we are given a summary of the list
summary(AP.data)

# You can see that there are 7 different data frames
# These data frames match what is listed under Part 1 above

* We can use the dollar sign ($) to get data from a lower level in the list object

In [ ]:
# The following code isolates one of the dataframes in AP.data
# So here we are selecting one of the dataframes in the larger list of dataframes

AP.data.siteinfo = AP.data$site.info

In [ ]:
# We can use the same code as above to examine the new object
# Notice that now the output is a data frame
class(AP.data.siteinfo)

In [ ]:
# Data frames have rows and columns - just like an excel spreadsheet
# The rows in the data frame are each of the ausplot sites
# The columns in the data frame are the different properties at each of those sites
# When we ask for the summary of the data frame we are given all the column headings of the data frame 
summary(AP.data.siteinfo)

In [ ]:
# This is very messy so we can also look at the first 6 rows of the dataframe
head(AP.data.siteinfo)

In [ ]:
# We can also see how many plots there are in the data frame by asking how many rows there are with the following
nrow(AP.data.siteinfo)

### It is also useful to save data frames off as another format such as csv
### Use the code below to save the new data frame to csv
* Once you have saved it, open it from the menu on the left and look at it's contents

In [ ]:
# Save an AusPlots derived Data Frame (generated for pre-processing), using 'write.csv'
# =====================================================================================
# Provide Path for Directory where data will be stored
file.path = "workspace"
# Create Name of the file to be stored
file.name = "site.info.eg1.csv" #############  TYPE HERE WHAT YOU WANT TO CALL YOUR FILE
# Save the Basal Area data to a Text File with columns separated by tabs
write.csv(AP.data.siteinfo, paste(file.path, file.name, sep="/"))

## Example 2: In this example we extract data for Adelaide and its sourrounding area

* The following code extracts data for Adelaide (34.92866S  138.59863E) and its sourrounding area
* This time the code puts the extracted data into the 'list object' called 'AP.data.AdelReg'

In [ ]:
# 'site_info', 'veg.vouchers', and 'veg.PI' data retrived for Adelaide (34.92866S  138.59863E) and its sourrounding area
# Notice that we do not ask for structural_summaries, basal.wedge, soil_subsites, soil_bulk_density or soil_character like we did in the first example
AP.data.AdelReg = get_ausplots(bounding_box=c(138.1, 139.1, -34.5, -35.5))

In [ ]:
# Using the same commands that we used before explore the new output

summary(AP.data.AdelReg)

In [ ]:
# Make a separate data frame of veg.vouch 
# Remember the veg.vouch dataframe details the different species found at the site 

AP.data.AdelReg.vegvouch = AP.data.AdelReg$veg.vouch

In [ ]:
# Using the same commands that we used before explore the new output
class(AP.data.AdelReg.vegvouch)  
summary(AP.data.AdelReg.vegvouch)

In [ ]:
# Again, examine the first 6 rows of the dataframe 
head(AP.data.AdelReg.vegvouch)

### Use the code below to save the new data frame to csv

* Open the csv and look at the types of information in it, note that it is considerably different from the site_info.
* Why is this?

In [ ]:
# Save an AusPlots derived Data Frame (generated for pre-processing), using 'write.csv'
# =====================================================================================
# Provide Path for Directory where data will be stored
file.path = "workspace"
# Create Name of the file to be stored
file.name = "veg.vouch.eg2.csv" #############  TYPE HERE WHAT YOU WANT TO CALL YOUR FILE
# Save the Basal Area data to a Text File with columns separated by tabs
write.csv(AP.data.AdelReg.vegvouch, paste(file.path, file.name, sep="/"))

## Example 3A: Get data for a transect in South Australia

In [ ]:
# make a list of sites
transect_list <- c('SATEYB0001', 'SATEYB0002', 'SATFLB0001', 'SATFLB0002', 'SATFLB0003', 'SATFLB0004', 'SATFLB0005', 'SATFLB0006', 
                   'SATFLB0007', 'SATFLB0008', 'SATFLB0009', 'SATFLB0010', 'SATFLB0011', 'SATFLB0012', 'SATFLB0013', 'SATFLB0014', 
                   'SATFLB0015', 'SATFLB0016', 'SATFLB0017', 'SATFLB0018', 'SATFLB0019', 'SATFLB0020', 'SATFLB0021', 'SATFLB0022', 
                   'SATFLB0023', 'SATFLB0024', 'SATFLB0025', 'SATFLB0026', 'SATFLB0027', 'SATFLB0028', 'SATKAN0001', 'SATKAN0002', 
                   'SATKAN0003', 'SATKAN0004', 'SATSTP0001', 'SATSTP0002', 'SATSTP0003', 'SATSTP0004', 'SATSTP0005', 'SATSTP0006', 
                   'SATSTP0007', 'SATSTP0008'
                  )

In [ ]:
# Here we filter by variables in the 'veg.PI' data frame
AP.data.transect = get_ausplots( my.Plot_IDs=transect_list, 
                       structural_summaries=TRUE, 
                       basal.wedge=TRUE)

In [ ]:
summary(AP.data.transect)


In [ ]:
# For a number of reasons we want to remove three unique sites (SATFLB0023-58712, SATFLB0024-58713, SATFLB0025-58714) from our analysis 
# Here we remove those sites and we will use this later in the analysis

AP.data.transect.site_info <- AP.data.transect$site.info
AP.data.transect.site_info <-AP.data.transect.site_info[!(AP.data.transect.site_info$site_unique=="SATFLB0023-58712" | 
                                      AP.data.transect.site_info$site_unique=="SATFLB0024-58713" | 
                                      AP.data.transect.site_info$site_unique=="SATFLB0025-58714"),]

In [ ]:
# Now we are going to limit the number of columns to just the information that we want
AP.data.site_coord = data.frame(site_unique=AP.data.transect.site_info$site_unique, 
                               latitude=AP.data.transect.site_info$latitude,
                               longitude=AP.data.transect.site_info$longitude
                              )

### The code below saves the dataframe that you just made as a csv file
* Once you have saved the dataframe, open it from the menu on the left and examine the contents

* **How does this new csv compare with "site.info.eg1.csv" from example 1?**
* **Why is this so?**

In [ ]:
# Save an AusPlots derived Data Frame using 'write.csv'
# =====================================================================================
# Provide Path for Directory where data will be stored
file.path = "workspace"
# Create Name of the file to be stored
file.name = "AP.data.site_coord.csv" #############  TYPE HERE WHAT YOU WANT TO CALL YOUR FILE
# Save the veg cover data to a Text File with columns separated by tabs
write.csv(AP.data.site_coord, paste(file.path, file.name, sep="/"))

# Vegetation Cover data by Height
## `single_cover_value` function

The `single_cover_value` function in the `auplotsR` package calculates Vegetation Cover Values for particular Growth Form Types and/or Height Thresholds per Site from Raw AusPlots Vegetation Point Intercept data. The `growth_form_table` function can also be used to calculate Cover Values for all Vegetation Growth Form Types; however, `single_cover_value` can perform these computations for:
* Particular vegetation growth form types (i.e. for individual growth forms or any combination of growth form types).
* Vegetation higher that a specified height threshold
* Vegetation with any combination of growth form types and minimum height

Specifically `single_cover_value` takes the following inputs via its arguments:
* `veg.PI`: Raw Vegetation Point Intercept data from AusPlots. A veg.PI data frame generated by the `get_ausplots` function (see above).
* `in_canopy_sky`: Method used to calculate Cover. A logical value that indicates whether to use in ‘canopy sky hits’ (i.e. calculate ‘opaque canopy cover’) or ‘projected foliage cover’. The default value, ‘FALSE’, calculates ‘projected foliage cover’. To calculate ‘opaque canopy cover’ the argument must be set to ‘TRUE’.
* `by.growth_form`: Whether to calculate Cover for a Subset by Growth Form type. A logical value that indicates whether to subset by growth form type. The default, ‘TRUE’, calculates cover for the growth form types specified in the argument ‘my.growth_forms’ (see next). If set to ‘FALSE’, cover calculations are conducted only for the vegetation sub-set by a provided Minimum Height Threshold.
* `my.growth_forms`: Growth Form Types used to Subset Data used for the Cover Calculations. A character vector specifying the growth form types to subset the data used for the cover calculations. Any combination of growth form types can be used. The default, ‘c("Tree/Palm", "Tree Mallee")’, is set to represent trees. It applies only when ‘by.growth_form=TRUE’; otherwise, this argument is ignored and only height sub-setting is applied.
* `min.height`: Minimum Height Threshold used to Subset Data used for the Cover Calculations. A numeric value indicating the minimum height (in metres) of the vegetation to be included in the subset of the data used for the cover calculations. A height must be always provided. The default, ‘5’, is set up for a cover of trees. It can be set to ‘0’ to ignore height and thus include any plant hit. If set to a ‘negative number’, it will return nonsensical output.

The `single_cover_value` function returns a data frame with two columns. The data frame rows correspond to unique sites, while the two columns correspond to the unique site and the percentage cover for the requested subset of vegetation (e.g. “Tree/Palm” higher than '5' metres).
 
When `by.growth_form = FALSE` and `min.height = 0`, the output is nearly the same as the green cover fraction returned by the `fractional_cover` function (see above). The values can differ because ‘fractional_cover’ applies a ‘height rule’ in which the highest intercept at a given point is taken, whereas ‘single_cover_value’ finds any green cover. For example, when dead trees overhang green understorey the values returned by both functions can differ. For general cover purposes, using ‘fractional_cover’ is recommended.  ‘single_cover_value’ is best suited to calculate cover subset by height and growth form.


## Example 3B: Explore veg cover for the transect in South Australia using the single cover value function

### Here we will extract a dataframe that contains the percentage cover for all sites the data across the transect
### Notice that we are asking for all the data (min.height=0)

In [ ]:
# Create a dataframe of all Vegetation Cover > 0m
AP.data.transect.vegPI.gt0 = single_cover_value(AP.data.transect$veg.PI, min.height=0)

In [ ]:
# Join the new vegetation dataframe with the AP.data.site_coord dataframe
AP.data.transect.vegPI.gt0 <- merge(AP.data.transect.vegPI.gt0, AP.data.site_coord, by = 'site_unique')

In [ ]:
head(AP.data.transect.vegPI.gt0)

### Save the dataframe as a csv

In [ ]:
# Save an AusPlots derived Data Frame using 'write.csv'
# =====================================================================================
# Provide Path for Directory where data will be stored
file.path = "workspace"
# Create Name of the file to be stored
file.name = "trans.VC.gt0.csv" #############  TYPE HERE WHAT YOU WANT TO CALL YOUR FILE
# Save the veg cover data to a Text File with columns separated by tabs
write.csv(AP.data.transect.vegPI.gt0, paste(file.path, file.name, sep="/"))

#### Here we will also get the percentage cover, but now only for vegetation with a minimum height of 2m (min.height=2)

In [ ]:
# Create a dataframe of Vegetation Cover > 2m
AP.data.transect.vegPI.gt2 = single_cover_value(AP.data.transect$veg.PI, min.height=2)

In [ ]:
# Join the new vegetation dataframe with the AP.data.site_coord dataframe
AP.data.transect.vegPI.gt2 <- merge(AP.data.transect.vegPI.gt2, AP.data.site_coord, by = 'site_unique')

### Save the dataframe as CSV

In [ ]:
# Save an AusPlots derived Data Frame using 'write.csv'
# =====================================================================================
# Provide Path for Directory where data will be stored
file.path = "workspace"
# Create Name of the file to be stored
file.name = "trans.VC.gt2.csv" #############  TYPE HERE WHAT YOU WANT TO CALL YOUR FILE
# Save the veg cover data to a Text File with columns separated by tabs
write.csv(AP.data.transect.vegPI.gt2, paste(file.path, file.name, sep="/"))

## Now we can also join the dataframes together

In [ ]:
# Combine the results into a single data frame 
# -----------------------------------------------------------------
AP.data.VC.Height = data.frame(site_unique=AP.data.transect.site_info$site_unique, 
                               VCF.gt0=AP.data.transect.vegPI.gt0$percentCover, 
                               VCF.gt2=AP.data.transect.vegPI.gt2$percentCover, 
                               latitude=AP.data.transect.site_info$latitude,
                               longitude=AP.data.transect.site_info$longitude
                              )
head(AP.data.VC.Height)

### Save the combined dataframe as a CSV
* Open the three csv files from the above exercises and compare their contents
* **What is the difference?**

In [ ]:
# Save an AusPlots derived Data Frame using 'write.csv'
# =====================================================================================
# Provide Path for Directory where data will be stored
file.path = "workspace"
# Create Name of the file to be stored
file.name = "VCF.Height.csv" #############  TYPE HERE WHAT YOU WANT TO CALL YOUR FILE
# Save the veg cover data to a Text File with columns separated by tabs
write.csv(AP.data.VC.Height, paste(file.path, file.name, sep="/"))

## Example 3C: Explore vegetation fractional cover
In the above exercises you looked at the total cover over different sites.
However, we can also extract the vegetation cover by fractional cover classes. <br><br>
Fractional cover classes are:
* photosynthetic vegetation
* non-photosynthetic vegetation 
* bare ground

In [ ]:
# Create a dataframe with fractional cover
# You will get an error message, this is just telling you that you have some missing data
AP.data.transect.FC = fractional_cover(AP.data.transect$veg.PI)

In [ ]:
# Join the new vegetation dataframe with the AP.data.site_coord dataframe
AP.data.transect.FC <- merge(AP.data.transect.FC, AP.data.site_coord, by = 'site_unique')

In [ ]:
head(AP.data.transect.FC)

### Join all the dataframes from Example 3 into a singe file

In [ ]:

AP.data.FC.save = data.frame(site_unique=AP.data.transect.site_info$site_unique, 
                               VCF.gt0=AP.data.transect.vegPI.gt0$percentCover, 
                               VCF.gt2=AP.data.transect.vegPI.gt2$percentCover, 
                               fc.bare=AP.data.transect.FC$bare,
                               fc.brown=AP.data.transect.FC$brown,
                               fc.green=AP.data.transect.FC$green,
                               latitude=AP.data.transect.site_info$latitude,
                               longitude=AP.data.transect.site_info$longitude
                              )

### Save the new data frame as a unique CSV file
* **Compare the new csv file with the previous files**
* **Do you notice anything different?**

In [ ]:
# Save an AusPlots derived Data Frame using 'write.csv'
# =====================================================================================
# Provide Path for Directory where data will be stored
file.path = "workspace"
# Create Name of the file to be stored
file.name.sting = "trans.FC.csv" #############  TYPE HERE WHAT YOU WANT TO CALL YOUR FILE
# Save the veg cover data to a Text File with columns separated by tabs
write.csv(AP.data.FC.save, paste(file.path, file.name.sting, sep="/"))

# Species occurrence matrices
In this section, we will explore to how to obtain and use species occurrence data from AusPlots raw data. In particular, we will examine species cover/abundance, species presence/absence, multiple indices of species diversity, and rank-abundance plots for the sites in the 5 most sampled bioregions. 

The first step to work with species-level AusPlots data is to create a species occurrence matrix. The 'species_table' function in the 'ausplotsR' package can be used to effortlessly create this type of matrix. This function takes a data frame of individual raw point intercept hits (i.e. a 'veg.PI' data frame) generated using the 'get_ausplots' function and returns a 'species against sites' matrix. Four metrics can be selected to score species occurrence: 

 * Presence/Absence: Set by the argument 'm_kind = PA'.
 * Percent Cover: Based on total frequency of hits. This is the most commonly used metric. Set by the argument 'm_kind = percent_cover'.
 * Frequency: Based on proportional frequencies of presence on the 10 individual transects within a plot. Set by the argument 'm_kind = freq'. It can be a measure of importance for low cover species.
## Example 4: Explore matricies

#### In this following cell you will create a dataframe that provides presence abcence data for each species 

In [ ]:
# species precence absence (PA)
AP.data.transect.vegPI.PA = species_table(AP.data.transect$veg.PI, m_kind="PA")
AP.data.transect.vegPI.PA$site_unique <- rownames(AP.data.transect.vegPI.PA)

In [ ]:
# Join the new vegetation dataframe with the AP.data.site_coord dataframe
AP.data.transect.vegPI.PA <- merge(AP.data.transect.vegPI.PA, AP.data.site_coord, by = 'site_unique')

### Save the new data frame as a unique CSV file
* Examine the CSV and answer the following questions. You may want to download the CSV to explore this fully.
* **How many times is Acacia pycnantha observed across the transect?**
* **What species is observed the most?**
* **What site has the most species?**

In [ ]:
# Save an AusPlots derived Data Frame using 'write.csv'
# =====================================================================================
# Provide Path for Directory where data will be stored
file.path = "workspace"
# Create Name of the file to be stored
file.name = "sp_table_pa.csv" #############  TYPE HERE WHAT YOU WANT TO CALL YOUR FILE
# Save the veg cover data to a Text File with columns separated by tabs
write.csv(AP.data.transect.vegPI.PA, paste(file.path, file.name, sep="/"))

# The end